In [0]:
import os
# changing directory to Google Drive Folder (when using Google Colab)
os.chdir('/content/drive/My Drive/ANN/hw1')

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.base import TransformerMixin

## Part 1 and 2

loding and preprocessing the data

In [139]:
# loading dataset
dataset_path = "dataset/adult.data"

# using ',\s' as seprator because we have '?,' in our dataset
# converting '?' to NaN
df = pd.read_csv(dataset_path, header=None, sep=',\s', na_values=["?"])

# Define dataset headers
df.columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship",
          "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "label"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [140]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [0]:
df_without_nan = DataFrameImputer().fit_transform(df)

In [0]:
# prepare input data
def one_hot_input(x_train):
	ohe = OneHotEncoder()
	ohe.fit(x_train)
	x_train_enc = ohe.transform(x_train)
	return x_train_enc

# prepare target
def one_hot_output(y_train):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	return y_train_enc

In [0]:
# choosing x_train and y_train
x_train = df_without_nan.loc[:,'age':'native-country'].values
y_train = df_without_nan['label'].values


x_train_enc = prepare_inputs(x_train)
y_train_enc = prepare_targets(y_train)

In [152]:
print(f"x_train shape: {x_train.shape}")
print(f"x_train_enc shape: {x_train_enc.shape}")
print()
print(f"y_train shape: {y_train.shape}")
print(f"y_train_enc shape: {y_train_enc.shape}")

x_train shape: (32561, 14)
x_train_enc shape: (32561, 22141)

y_train shape: (32561,)
y_train_enc shape: (32561,)



# Part 3 - Model


In [0]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(units=14, input_dim=x_train_enc.shape[1]),
                                    tf.keras.layers.Dense(units=128, activation='relu'),
                                    tf.keras.layers.Dense(units=1, activation='sigmoid')

])

In [0]:
# compiling model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

In [150]:
model.fit(x_train_enc, y_train_enc, epochs=10)#, validation_split=0.1)#, callbacks=[es_callback, tb_callback])

Epoch 1/10
1018/1018 [==============================] - 3s 3ms/step - loss: 0.3233 - accuracy: 0.8513
Epoch 2/10
1018/1018 [==============================] - 3s 3ms/step - loss: 0.2339 - accuracy: 0.8961
Epoch 3/10
1018/1018 [==============================] - 3s 2ms/step - loss: 0.1597 - accuracy: 0.9325
Epoch 4/10
1018/1018 [==============================] - 3s 2ms/step - loss: 0.1225 - accuracy: 0.9465
Epoch 5/10
1018/1018 [==============================] - 2s 2ms/step - loss: 0.1023 - accuracy: 0.9525
Epoch 6/10
1018/1018 [==============================] - 3s 2ms/step - loss: 0.0888 - accuracy: 0.9561
Epoch 7/10
1018/1018 [==============================] - 3s 3ms/step - loss: 0.0754 - accuracy: 0.9624
Epoch 8/10
1018/1018 [==============================] - 3s 3ms/step - loss: 0.0657 - accuracy: 0.9676
Epoch 9/10
1018/1018 [==============================] - 3s 3ms/step - loss: 0.0562 - accuracy: 0.9728
Epoch 10/10
1018/1018 [==============================] - 3s 2ms/step - loss: 0.050

In [36]:
# checking model summary
model.summary()

ValueError: ignored